In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
DATASET_PMC_FULL = './gdrive/Shareddrives/DATASETS/PMC-Sents-FULL/'
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

In [ ]:
!pip install transformers -q
!pip install datasets -q

     |████████████████████████████████| 4.7 MB 8.2 MB/s 
     |████████████████████████████████| 101 kB 10.4 MB/s 
     |████████████████████████████████| 596 kB 51.0 MB/s 
     |████████████████████████████████| 6.6 MB 43.0 MB/s 
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 1.3 MB 45.4 MB/s 
     |████████████████████████████████| 365 kB 7.4 MB/s 
     |████████████████████████████████| 212 kB 50.4 MB/s 
     |████████████████████████████████| 141 kB 34.2 MB/s 
     |████████████████████████████████| 115 kB 49.3 MB/s 
     |████████████████████████████████| 127 kB 52.7 MB/s 


In [ ]:
import pandas as pd

In [ ]:
df_test = pd.read_parquet(DATASET_PMC_FULL + 'test_full_text_span1.parquet')

In [ ]:
df_test.columns

Index(['id', 'text', 'label_id'], dtype='object')

In [ ]:
# same text but with different labels
print(f'{len(df_test.index)}')
mask = df_test.groupby('text')['label_id'].transform('nunique') > 1
df_test = df_test[~mask].copy()
print(f'{len(df_test.index)}')

17310
17310


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoConfig, AutoTokenizer

model_path = './gdrive/Shareddrives/MODELS/pmc_full_scibert_classification'
config = AutoConfig.from_pretrained(model_path)

tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
model = AutoModelForSequenceClassification.from_pretrained(model_path, config=config)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = model.to(device)
model.device

device(type='cuda', index=0)

In [ ]:
from transformers import TextClassificationPipeline

tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0, **tokenizer_kwargs)

In [ ]:
predictions = pipe(list(df_test['text']))

In [ ]:
y_pred = []
for pred in predictions:
  y_pred.append(config.label2id[pred['label']])

In [ ]:
y_test_true = list(df_test['label_id'])

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score

output_dict = {}
output_dict['f1_macro'] = f1_score(y_test_true, y_pred, average='macro')
output_dict['f1_micro'] = f1_score(y_test_true, y_pred, average='micro')
output_dict

{'f1_macro': 0.5528833249935268, 'f1_micro': 0.6997111496244945}

In [ ]:
config.id2label

{0: 'background', 1: 'objective', 2: 'method', 3: 'result', 4: 'other'}

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test_true, y_pred, target_names=config.id2label.values()))

              precision    recall  f1-score   support

  background       0.72      0.84      0.77      9289
   objective       0.68      0.29      0.41       603
      method       0.67      0.70      0.68      4096
      result       0.72      0.47      0.57      2039
       other       0.59      0.23      0.33      1283

    accuracy                           0.70     17310
   macro avg       0.67      0.51      0.55     17310
weighted avg       0.69      0.70      0.68     17310

